In [1]:
### imports
import json
from copy import deepcopy

from nltk.tokenize import sent_tokenize, TweetTokenizer
tokenizer = TweetTokenizer()
from string import punctuation
import re
import regex
from nltk.corpus import stopwords
stopW = stopwords.words('english')
# stopW = ['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', 'her', 'here', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in', 'into', 'is', 'isn', "isn't", 'it', "it's", 'its', 'itself', 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she', "she's", 'should', "should've", 'shouldn', "shouldn't", 'so', 'some', 'such', 't', 'than', 'that', "that'll", 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'there', 'these', 'they', 'this', 'those', 'through', 'to', 'too', 'under', 'until', 'up', 've', 'very', 'was', 'wasn', "wasn't", 'we', 'were', 'weren', "weren't", 'what', 'when', 'where', 'which', 'while', 'who', 'whom', 'why', 'will', 'with', 'won', "won't", 'wouldn', "wouldn't", 'y', 'you', "you'd", "you'll", "you're", "you've", 'your', 'yours', 'yourself', 'yourselves']
# print(sorted(stopW))

In [2]:
### a method to Read a json file with img captions and article titles and parse it into set of documents

def getDocs(sourcePath):
    medDocs = []
    sourceFile = open(sourcePath, 'r')

    count = 1
    for line in sourceFile:
        medDocs.append(json.loads(line))
        count += 1

    sourceFile.close()
    
    # for k in range(1,count):
    #     print(str(k) + " : " + medDocs[k]['docId'])
    return medDocs


sourcePath = 'assets/medImages.jl'
medDocs = getDocs(sourcePath)
# print("Total number of figures: " + str(len(medDocs)))
# medDocs[0]['caption']
# medDocs[0]['caption'][1]

In [3]:
# method that cleans the title and caption text in each document
# the result is tokenized titles and caption without unwanted brackets, numbers, spaces, tags, unicode charts;
# result includes no punctuation and stopwards and tokens with length less than 2 symbols
# all tokens are lower case


def preprocessContent(content):
    sentences = sent_tokenize(content)
    tokens = []
    for sent in sentences:
        # removes unicode
        cleanedTokens = re.sub(r'[^\x00-\x7F]+', ' ', sent)
        # removes menions
        cleanedTokens = re.sub(r'@\w+', '', cleanedTokens)
        # removes numbers
        cleanedTokens = re.sub('[0-9]+', '', cleanedTokens)
        # removes nested brackets 
        cleanedTokens = regex.sub(r'\([^()]*+(?:(?R)[^()]*)*+\)', '', cleanedTokens) 
        # removes nested currly brackets
        cleanedTokens = regex.sub('\{(?:[^}{]|\{[^}{]*\})*\}', '', cleanedTokens) 
        # removes html tags
        cleanedTokens = re.sub('\<+/*\w*/*\>+', '', cleanedTokens)
        # removes punctuation
        cleanedTokens = re.sub(r'[%s]' % re.escape(punctuation), ' ', cleanedTokens)
        # removes doubled spaces
        cleanedTokens = re.sub(r'\s{2,}', ' ', cleanedTokens)
        sentTokens = tokenizer.tokenize(cleanedTokens)
        # lower case each token and removes tokens with length less than 2
        sentTokens = [tok.lower() for tok in sentTokens if len(tok) >2]
        # removes stop words 
        sentTokens2 = [tok for tok in sentTokens if not tok.lower() in stopW ]
        tokens += sentTokens2
    return tokens


def cleanDocs(medDocs):
    cleanedDocs = deepcopy(medDocs)
    for i, doc in enumerate(cleanedDocs):
        doc['articleTitle'] = preprocessContent(doc['articleTitle'] )
        for j, cap in enumerate(doc['caption']):
            doc['caption'][j] = preprocessContent(cap)
    return cleanedDocs

cleanedDocs = cleanDocs(medDocs)

In [4]:
# print(len(cleanedDocs))
# print(medDocs[0]['caption'])
# print(cleanedDocs[0]['caption'])
# cleanedDocs[262]

# print(medDocs[0]['caption'])
# print(cleanedDocs[0]['caption'])

In [5]:
### create a bag of words 
def createBagsOfWOrds(corpus):
    bagOfWordsTitle = []
    bagOfWordsCaption = []
    for doc in corpus:
        for term in doc['articleTitle']:
            if term not in bagOfWordsTitle:
                bagOfWordsTitle.append(term)
        for cap in doc['caption']:
            for term in cap:
                if term not in bagOfWordsCaption:
                    bagOfWordsCaption.append(term)
    
    return bagOfWordsTitle, bagOfWordsCaption

bagOfWordsTitle, bagOfWordsCaption = createBagsOfWOrds(cleanedDocs)

# print(bagOfWordsTitle[:10])#, bagOfWordsCaption[:10])
# print(len(bagOfWordsTitle))
# print(len(bagOfWordsCaption))

In [11]:
def capDicts(corpus):
    captionsDict = {}
    captionCount = 0
    caption_idx = 0
    for i, doc in enumerate(corpus):
        for j in range(len(doc['caption'])):
            captionsDict[caption_idx] = i
            caption_idx += 1
        
    return captionsDict


captions_doc_dict = capDicts(cleanedDocs)
